# Example-Based Model

## Initialization of the Datasets


### Source Dataset

#### Storing the Tagalog Part of Speech Data Set

In [ ]:
import pandas as pd

# Read the tagalog POS dataset
tl_pos_data = pd.read_json('src/json data/tl_pos.json')

tl_pos_data.head(2)

#### Storing the Ilokano Part of Speech Data Set

In [ ]:
# Read the ilokano POS dataset
il_pos_data = pd.read_json('src/json data/il_pos.json')

il_pos_data.head(2)

#### Storing the Tagalog Part of Speech 

In [ ]:
temp_df = pd.DataFrame(tl_pos_data['POS'])

temp_df.columns = ['Tagalog POS']
temp_df.head()

#### Storing the Ilokano Part of Speech 

In [ ]:
temp_df['Ilokano POS'] = il_pos_data['POS']

temp_df.head()

In [ ]:
tl_sample_sen = temp_df['Tagalog POS'][1]
il_sample_sen = temp_df['Ilokano POS'][1]

tl_verb_list = []
il_verb_list = []
tl_to_il_verbs = []

for tl_word in tl_sample_sen:
   for il_word in il_sample_sen:
      if tl_word == il_word and tl_word == 'VB':
         """
         if the tagalog word has a pos 'VB' and the ilokano word has a pos 'VB'
         """
         
         tl_temp_index = tl_sample_sen.index(tl_word)
         il_temp_index = il_sample_sen.index(il_word)
         """
         gets the index of those words
         """
         il_temp_sen = il_pos_data['Tokenized'][1]
         tl_temp_verb = tl_pos_data['Tokenized'][1][tl_temp_index]
         
         try:
            prev_pos = il_sample_sen[il_temp_index - 1]
         except (ValueError, IndexError):
            prev_pos = None
         """
         getting the previous pos of the ilokano word
         """   
         
         if prev_pos == 'DT':
            """
            if the ilokano word has a pos 'DT' before it
            """
            il_temp_verb = il_temp_sen[il_temp_index - 1] + ' ' + il_temp_sen[il_temp_index]
         else:
            il_temp_verb = il_temp_sen[il_temp_index]

tl_verb_list.append(tl_temp_verb)
il_verb_list.append(il_temp_verb)
il_verb_list.append('aramid')
tl_to_il_verbs.append(tl_verb_list)
tl_to_il_verbs.append(il_verb_list)
   
"""
append the tagalog word to the list
"""
print(tl_to_il_verbs)           
              